In [1]:
# use ollama https://github.com/ollama/ollama?tab=readme-ov-file
# 請先把ollama打開
# terminal: ollama pull llama3
# 另外可以讀一下這個
# https://myapollo.com.tw/blog/langchain-tutorial-retrieval/

In [2]:
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama


In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})

/opt/miniconda3/envs/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/miniconda3/envs/.venv/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/opt/miniconda3/envs/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed i

In [4]:
file_path = "/Users/mac/Documents/code/pubmedtool/16402132.pdf"
loader = file_path.endswith(".pdf") and PyPDFLoader(file_path) or TextLoader(file_path)

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100) 
texts = loader.load_and_split(splitter)

In [6]:
# vectorstore = Chroma.from_documents(texts, embeddings, persist_directory ='db')
vectorstore = Chroma.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever()

In [8]:
llm = Ollama(model="llama3")


In [9]:
query = '''what is the title of paper?'''

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

chain.invoke({"input": query})